In [17]:
from libraries import *
from parameters import *
import statsmodels.api as sm
import sklearn.feature_selection
import statsmodels.api as sm
import sklearn.feature_selection
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')
from statannot import add_stat_annotation
from numpy.random import seed
from numpy.random import randint
import statsmodels.formula.api as smf

%load_ext rpy2.ipython

In [ ]:
# data = sm.datasets.get_rdataset("dietox", "geepack").data
# md = smf.mixedlm("Weight ~ Time", data, groups=data["Pig"], re_formula="~Time")
# mdf = md.fit(method=["lbfgs"])
# mdf.random_effects
# mdf.summary()

In [2]:
os.getcwd()
os.chdir(projectDir)

In [3]:
adataC = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_08122020_PerGENE.h5ad')


In [ ]:
# adataC.obs.loc[adataC.obs.leiden.isin(['3']), "subCellType"] = "MacDC"
# adataC.obs.loc[adataC.obs.leiden.isin(['8']), "subCellType"] = "DC1"
# adataC.obs.loc[adataC.obs.leiden.isin(['5']), "subCellType"] = "MReg"

In [4]:
covariates = adataC.uns['feature_barcode_names_filtered_GENES']
covariates = covariates[covariates != "GENE_CONTROL_"]

In [5]:
len(covariates)

1031

In [6]:
guideMatrix = adataC.obs[covariates]
guideMatrix.shape
guideMatrix.rename(columns = {'GENE_Rnf8-cmtr1_':'GENE_Rnf8cmtr1_'}, inplace = True) 
guideMatrix.rename(columns = {'GENE_Siah1-ps1_':'GENE_Siah1ps1_'}, inplace = True) 
guideMatrix.rename(columns = {'GENE_Siah1-ps2_':'GENE_Siah1ps2_'}, inplace = True) 


In [7]:
## Add covariates that should be corrected for
df = adataC.obs[["n_genes", "mt_frac", "leiden" ]]

guideMatrix = guideMatrix.join(df)
guideMatrix.shape

(242938, 1034)

In [8]:
expressionMatrix = adataC.X

In [13]:
controlExpressionMat = expressionMatrix[adataC.obs["GENE_CONTROL_"] == 1,]
controlGuideMat = guideMatrix.loc[adataC.obs["GENE_CONTROL_"] == 1,]


In [14]:
controlExpressionMat = controlExpressionMat[0:5000,:]
controlGuideMat = controlGuideMat.iloc[0:5000,:]

In [15]:
geneExpressionMat = expressionMatrix[adataC.obs[covariates[0:100]].sum(axis=1) > 0,]
geneGuideMat = guideMatrix.loc[adataC.obs[covariates[0:100]].sum(axis=1) > 0,]

In [ ]:
# geneExpressionMat = expressionMatrix[adataC.obs["GENE_Aamp_"] == 1,]
# geneGuideMat = guideMatrix.loc[adataC.obs["GENE_Aamp_"] == 1,]

In [ ]:
allExpMat = np.concatenate((controlExpressionMat, geneExpressionMat))
allGuideMat = controlGuideMat.append(geneGuideMat)

In [21]:
allVars = covariates[0:100]
allVars

array(['GENE_4921501E09Rik_', 'GENE_4930595M18Rik_',
       'GENE_8030462N17Rik_', 'GENE_Aamp_', 'GENE_Abtb1_', 'GENE_Abtb2_',
       'GENE_Acaca_', 'GENE_Ago2_', 'GENE_Ahctf1_', 'GENE_Ahr_',
       'GENE_Akt1_', 'GENE_Ambra1_', 'GENE_Amfr_', 'GENE_Anapc10_',
       'GENE_Anapc11_', 'GENE_Anapc13_', 'GENE_Anapc1_', 'GENE_Anapc2_',
       'GENE_Anapc4_', 'GENE_Anapc5_', 'GENE_Anapc7_', 'GENE_Ankfy1_',
       'GENE_Apaf1_', 'GENE_Apbb1_', 'GENE_Apc_', 'GENE_Ar_',
       'GENE_Arel1_', 'GENE_Arih1_', 'GENE_Arih2_', 'GENE_Armc8_',
       'GENE_Arnt_', 'GENE_Arpc1a_', 'GENE_Arpc1b_', 'GENE_Arrb1_',
       'GENE_Asb11_', 'GENE_Asb12_', 'GENE_Asb14_', 'GENE_Asb15_',
       'GENE_Asb1_', 'GENE_Asb2_', 'GENE_Asb3_', 'GENE_Asb4_',
       'GENE_Asb6_', 'GENE_Asb7_', 'GENE_Asb9_', 'GENE_Atg16l1_',
       'GENE_Atg16l2_', 'GENE_Atg5_', 'GENE_Atxn3_', 'GENE_Aurka_',
       'GENE_Bach1_', 'GENE_Bard1_', 'GENE_Baz1a_', 'GENE_Baz1b_',
       'GENE_Bcl6_', 'GENE_Bfar_', 'GENE_Bid_', 'GENE_Birc2_',
     

In [ ]:
all_columns = "+".join(allVars[~allVars.isin(["leiden"])])
all_columns = all_columns + "+n_genes"+"+mt_frac"

In [ ]:
all_columns

In [ ]:
allGuideMat

In [ ]:
allVars

In [ ]:
%%R -i allVars  -w 8 -h 4 -u in
xVars = paste0(allVars, collapse="+")
paste0("y ~ ", xVars, "+ n_genes +  mt_frac + (1+", xVars,"|leiden)")

In [24]:
%%R -i controlExpressionMat,controlGuideMat,geneExpressionMat,geneGuideMat,allVars  -w 8 -h 4 -u in
library("lme4")

selCells = which(geneGuideMat[,"GENE_Aamp_"]> 0)

selExpressionMat <- geneExpressionMat[selCells,]
selGuideMat <- geneGuideMat[selCells,]

selGuideMat["y"] = selExpressionMat[,1]

mixed.lmer <- lmer(formula(paste0("y ~ ", "GENE_Aamp_", "+ n_genes +  mt_frac + (1+", "GENE_Aamp_","|leiden)")), data = selGuideMat)
modelSum <- summary(mixed.lmer)
modelSum


R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

R[write to console]: boundary (singular) fit: see ?isSingular



Linear mixed model fit by REML ['lmerMod']
Formula: y ~ GENE_Aamp_ + n_genes + mt_frac + (1 + GENE_Aamp_ | leiden)
   Data: selGuideMat

REML criterion at convergence: 236.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.2578 -0.5988 -0.3619 -0.1339  3.5434 

Random effects:
 Groups   Name        Variance  Std.Dev. Corr 
 leiden   (Intercept) 0.0004572 0.02138       
          GENE_Aamp_  0.0004560 0.02135  -1.00
 Residual             0.9759641 0.98791       
Number of obs: 81, groups:  leiden, 7

Fixed effects:
              Estimate Std. Error t value
(Intercept) -0.9150815  0.5702018  -1.605
n_genes      0.0006497  0.0002703   2.404
mt_frac     -0.3294774  6.0180677  -0.055

Correlation of Fixed Effects:
        (Intr) n_gens
n_genes -0.738       
mt_frac -0.657  0.015
fit warnings:
fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
Some predictor variables are on very different scales: consider rescaling
optimizer (nloptwrap) convergence

In [ ]:
allExpMat.shape

In [ ]:
allGuideMat["y"] = np.array(allExpMat)[:,0]


In [ ]:
est = smf.mixedlm("y~GENE_Aamp_+n_genes+mt_frac", allGuideMat, groups=allGuideMat["leiden"], missing='drop')

mdf = est.fit(method=["bfgs"])

mdf.summary()

In [ ]:
%%R -i allGuideMat -w 8 -h 4 -u in

library("lme4")

mixed.lmer <- lmer(formula("y ~ GENE_Aamp_ + n_genes +  mt_frac + (1+GENE_Aamp_|leiden)"), data = allGuideMat)
h <- summary(mixed.lmer)
h

#ranef(mixed.lmer)

In [ ]:
%%R  -w 8 -h 4 -u in
data.frame(h$varcor)


In [ ]:
%%R  -w 8 -h 4 -u in
attributes(h)

In [ ]:
%%R  -w 8 -h 4 -u in
h <- data.frame(ranef(mixed.lmer))
h 

In [ ]:
%%R  -w 8 -h 4 -u in
h$condval <- h$condval + 5.979e-02

In [ ]:
%%R  -w 8 -h 4 -u in
h

In [ ]:
%%R  -w 8 -h 4 -u in
coef(mixed.lmer)

In [ ]:
%%R  -w 8 -h 4 -u in
h$vcov

In [ ]:
guideMatrix.columns[0:1031]

In [ ]:
pd.Series(adataC.var_names).to_csv("/home/beraslan/jovian-work/analysisSingle/Gene_names.csv", index=False)

In [ ]:
allVars = pd.Series(guideMatrix.columns) 
allVars

In [ ]:
all_columns = "+".join(allVars[~allVars.isin(["leiden"])])


In [ ]:
my_formula = "y~" + all_columns
my_formula

In [ ]:
guideMatrix["y"] = np.array(expressionMatrix)[:,1]


In [ ]:
est = smf.mixedlm("y~GENE_Aamp_", guideMatrix, groups=guideMatrix["leiden"], missing='drop',re_formula=("~GENE_Aamp_"))

In [ ]:
mdf = est.fit(method=["bfgs"])

In [ ]:
mdf.summary().tables

In [ ]:
mdf.random_effects

In [ ]:
est = smf.mixedlm(my_formula, guideMatrix, groups=guideMatrix["leiden"], missing='drop',re_formula=("~"+all_columns))

In [ ]:
mdf.random_effects

In [ ]:
for i in range(5760,6000,20):
    print(i)
    pvalDF = pd.DataFrame()
    coefDF = pd.DataFrame()
    
    for j in range(i,i+20):
        print(j)
        guideMatrix["y"] = np.array(expressionMatrix)[:,j]
        est = smf.mixedlm(my_formula, guideMatrix, groups=guideMatrix["leiden"], missing='drop')
        mdf = est.fit(method=["bfgs"])
        pvalDF[str(j)] = mdf.summary().tables[1]["P>|z|"]
        coefDF[str(j)] = mdf.summary().tables[1]["Coef."]
    
    coefDF.index = mdf.summary().tables[1].index
    pvalDF.index = mdf.summary().tables[1].index
    pvalDF.to_csv("pvalues_"+str(i)+"_"+str(i+20)+"_MixedEffect.csv")   
    coefDF.to_csv("coefs_"+str(i)+"_"+str(i+20)+"_MixedEffec.csv")   